In [1]:
# get trauma_preprocessed
import numpy as np
import pandas as pd
import sklearn

# Replace 'your_file.csv' with the path to your CSV file
trauma_preprocessed = pd.read_csv('trauma_preprocessed_final.csv')

# Display the first few rows of the DataFrame to confirm import
print(trauma_preprocessed.head())
print(trauma_preprocessed.shape)


   Unnamed: 0  onehot__SEX_1.0  onehot__SEX_2.0  onehot__SEX_3.0  \
0           0              0.0              1.0              0.0   
1           1              1.0              0.0              0.0   
2           2              1.0              0.0              0.0   
3           3              1.0              0.0              0.0   
4           4              0.0              1.0              0.0   

   onehot__ETHNICITY_1.0  onehot__ETHNICITY_2.0  onehot__TBIMIDLINESHIFT_1.0  \
0                    0.0                    1.0                          0.0   
1                    1.0                    0.0                          0.0   
2                    0.0                    1.0                          0.0   
3                    0.0                    1.0                          0.0   
4                    0.0                    1.0                          0.0   

   onehot__TBIMIDLINESHIFT_2.0  onehot__TBIMIDLINESHIFT_3.0  \
0                          1.0                 

# Model training
Will first use a small model trained on 10,000 units before full set.

In [2]:
# making a small dataset to test models easier
# Set random seed for reproducibility
np.random.seed(42)

# Number of rows to selecte
n = 10000

# Randomly select n rows
trauma_preprocessed_small= trauma_preprocessed.sample(n=n, random_state=42)

# Display the first few rows of the sampled DataFrame
print(trauma_preprocessed_small.shape)

(10000, 81)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix



# Define the preprocessing steps for categorical and continuous features

# temp solution just drop the cat features. dont seem physiologically important, lets prototype this
# there are several continuous vars that will need to be binarized or made into categoricals though



# Separate the features (X) and the target variable (y)
X = trauma_preprocessed_small.drop('transfusion', axis=1)
y = trauma_preprocessed_small['transfusion']



# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)






# Model tuning
Now that you got some good pre-processing done, you will need to do the model fine tuning. look at what each of them can do, and what is significant, etc. basically find the one that works the best given all this data you got.

In [15]:
# logistic regression
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# Add a constant (intercept) to the model
X_train_lr = sm.add_constant(X_train)


def stepwise_logistic_regression(X, y, threshold_in=0.05, threshold_out=0.10):
    """
    Perform stepwise logistic regression using AIC as the criterion for backward elimination.
    """
    initial_features = X.columns.tolist()  # List of all columns (features)
    while True:
        model = sm.Logit(y, X[initial_features])
        result = model.fit()
        
        # Get the highest p-value predictor
        p_values = result.pvalues
        max_p_value = p_values.max()
        
        if max_p_value > threshold_out:  # If the p-value is higher than the threshold, remove it
            excluded_feature = p_values.idxmax()  # Feature with the highest p-value
            print(f"Excluding {excluded_feature} with p-value {max_p_value}")
            initial_features.remove(excluded_feature)  # Remove the feature from the list
        else:
            break  # If no feature can be removed, stop the process
    
    # Final model
    final_model = sm.Logit(y, X[initial_features])
    final_result = final_model.fit()
    
    print("\nFinal model after stepwise selection:")
    print(final_result.summary())
    
    return final_result

# Perform stepwise logistic regression
final_model = stepwise_logistic_regression(X_train_lr, y_train)


# result = final_model.fit()

# # Print the p-values of each predictor
# print("P-values for each predictor:")
# print(result.pvalues)


         Current function value: 0.226092
         Iterations: 35
Excluding label__ETHNICITY with p-value 0.9999999999998618


c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.226092
         Iterations: 35
Excluding label__PROTDEV_AIRBAG_PRESENT with p-value 0.9813107137343111


c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.226092
         Iterations: 35
Excluding onehot__TBIMIDLINESHIFT_2.0 with p-value 0.9999997481068926
         Current function value: 0.226092
         Iterations: 35
Excluding onehot__TEACHINGSTATUS_1.0 with p-value 0.9999986076695293


...

         Current function value: 0.226092
         Iterations: 35
Excluding onehot__ETHNICITY_2.0 with p-value 0.9999986514544464
         Current function value: 0.226092
         Iterations: 35
Excluding onehot__SEX_1.0 with p-value 0.999999229140904


c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.226092
         Iterations: 35
Excluding onehot__SEX_3.0 with p-value 0.9979239033663566
Optimization terminated successfully.
         Current function value: 0.226104
         Iterations 8
Excluding label__PROTDEV_HELMET with p-value 0.9477956396169486
Optimization terminated successfully.
         Current function value: 0.226105
         Iterations 8
Excluding scaler__HOSPITALARRIVALDAYS with p-value 0.8754347874998852


c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.226107
         Iterations 8
Excluding label__RACE_UK with p-value 0.867583681714607
Optimization terminated successfully.
         Current function value: 0.226109
         Iterations 8
Excluding label__RACEOTHER with p-value 0.8709785088771107
Optimization terminated successfully.
         Current function value: 0.226111
         Iterations 8
Excluding label__PREHOSPITALCARDIACARREST with p-value 0.8437691324610596
Optimization terminated successfully.
         Current function value: 0.226113
         Iterations 8
Excluding label__DRGSCR_METHAMPHETAMINE with p-value 0.8424189157477222
Optimization terminated successfully.
         Current function value: 0.226116
         Iterations 8
Excluding label__DRGSCR_NONE with p-value 0.8581889865230554
Optimization terminated successfully.
         Current function value: 0.226118
         Iterations 8
Excluding scaler__RESPIRATORYRATE with p-value 0.7899055362066887


In [ ]:
# using final model to make predictions

# List of selected features (columns) after stepwise selection
selected_features = final_model.model.exog_names  # Get the names of the selected features from the final model
selected_features.remove('const')

# Make sure to drop the constant column if it's present in X_test (it should be)
X_test_selected = X_test[selected_features]
X_test_selected = sm.add_constant(X_test_selected) # add const back

# Predict on the test data using the final model with only selected features
predictions_prob = final_model.predict(X_test_selected)  # Get predicted probabilities

# Convert probabilities to binary class predictions (0 or 1)
predictions = (predictions_prob >= 0.5).astype(int)

# Generate the confusion matrix
cm = confusion_matrix(y_test, predictions)
print("\nConfusion Matrix:")
print(cm)



Confusion Matrix:
[[2695   36]
 [ 214   55]]


NameError: name 'accuracy_score' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix

# Extract values from the confusion matrix
TN, FP, FN, TP = cm.ravel()

# Calculate metrics
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) != 0 else 0  # Avoid division by zero
recall = TP / (TP + FN) if (TP + FN) != 0 else 0      # Avoid division by zero
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
sensitivity = recall  # Sensitivity is the same as Recall
specificity = TN / (TN + FP) if (TN + FP) != 0 else 0   # Avoid division by zero

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"F1-Score: {f1_score:.4f}")
#

Accuracy: 0.9167
Precision: 0.6044
Recall (Sensitivity): 0.2045
Specificity: 0.9868
F1-Score: 0.3056


In [22]:
# lets try stepwise logistic regression first
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import accuracy_score

# Initialize logistic regression model
log_reg = LogisticRegression()


# Forward Stepwise Selection using AIC
def forward_selection_aic(X, y):
    remaining_features = X.columns.tolist()
    selected_features = []
    best_aic = np.inf  # Start with a very high AIC to ensure the first model improves it
    
    while remaining_features:
        aic_with_added_feature = {}
        
        for feature in remaining_features:
            features_temp = selected_features + [feature]
            X_train_step = X[features_temp]
            X_train_step = sm.add_constant(X_train_step)  # Add constant term (intercept)
            model = sm.Logit(y, X_train_step)  # Logistic Regression model
            result = model.fit(disp=0)  # disp=0 suppresses output
            
            # Record AIC for this model
            aic_with_added_feature[feature] = result.aic
        
        # Find the feature that minimizes AIC
        best_feature = min(aic_with_added_feature, key=aic_with_added_feature.get)
        new_aic = aic_with_added_feature[best_feature]
        
        # If adding the best feature improves the AIC, add it to selected features
        if new_aic < best_aic:
            selected_features.append(best_feature)
            best_aic = new_aic
            remaining_features.remove(best_feature)
        else:
            break  # Stop if no improvement in AIC
    
    return selected_features


# Forward Stepwise Selection
def stepwise_selection(X, y, model, threshold_in=0.1, threshold_out = 0.05):
    initial_features = X.columns.tolist()
    selected_features = []
    
    while len(initial_features) > 0:
        # Fit the model with the current set of selected features
        best_pval = 1
        best_feature = None
        for feature in initial_features:
            # print(feature)
            # Fit model with one additional feature
            X_train_step = X[selected_features + [feature]]
            model.fit(X_train_step, y)
            # Using p-values from the logistic regression
            pval = model.score(X_train_step, y)
            if pval < best_pval:
                print(feature, pval)
                best_pval = pval
                best_feature = feature
        
        # If the best feature is statistically significant, add it to the model
        if best_pval < threshold_in:
            selected_features.append(best_feature)
            initial_features.remove(best_feature)
        else:
            break

    return selected_features

# Perform stepwise selection
selected_features = forward_selection_aic(X_train, y_train)

# Final model with selected features
X_train_selected = X_train[selected_features]
log_reg.fit(X_train_selected, y_train)

# Evaluate the model
X_test_selected = X_test[selected_features]
y_pred = log_reg.predict(X_test_selected)

accuracy = accuracy_score(y_test, y_pred)
print(f"Selected Features: {selected_features}")
print(f"Model Accuracy: {accuracy}")


c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\micha\anaconda3\envs\trauma\Lib\site-packag

Selected Features: ['scaler__ISS', 'label__HIGHESTACTIVATION', 'scaler__AgeYears', 'scaler__SBP', 'scaler__PULSERATE', 'label__PMGCSQ_NA', 'label__TRANSPORTMODE', 'label__PMGCSQ_INTUBATED', 'onehot__TBIMIDLINESHIFT_3.0', 'onehot__TEACHINGSTATUS_6.0', 'label__WHITE', 'label__INTERFACILITYTRANSFER', 'label__TM_GROUNDAMBULANCE', 'Unnamed: 0', 'scaler__TEMPERATURE', 'label__PRIMARYMETHODPAYMENT', 'onehot__TEACHINGSTATUS_1.0', 'label__DRGSCR_COCAINE', 'scaler__HOSPITALARRIVALHRS', 'label__GCSQ_UK', 'label__TBIPUPILLARYRESPONSE']
Model Accuracy: 0.9166666666666666


c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
X_train.columns.tolist()
(selected_features)

[]

In [ ]:
# Create and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predict the target variable for the test data
y_pred = knn.predict(X_test)

# Evaluate the classifier
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[2704   27]
 [ 246   23]]

Classification Report:
              precision    recall  f1-score   support

       False       0.92      0.99      0.95      2731
        True       0.46      0.09      0.14       269

    accuracy                           0.91      3000
   macro avg       0.69      0.54      0.55      3000
weighted avg       0.88      0.91      0.88      3000



In [ ]:
# now lets try decision tree classifiers
from sklearn import tree

y_binary = y.astype(int)

X_train, X_test, y_train_b, y_test_b = train_test_split(X_scaled, y_binary, test_size=0.3, random_state=42)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train_b)

clf_pred = clf.predict(X_test)

# Evaluate the classifier
print("Confusion Matrix:")
print(confusion_matrix(y_test_b, clf_pred))
print("\nClassification Report:")
print(classification_report(y_test_b, clf_pred))

Confusion Matrix:
[[2507  224]
 [ 174   95]]

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      2731
           1       0.30      0.35      0.32       269

    accuracy                           0.87      3000
   macro avg       0.62      0.64      0.62      3000
weighted avg       0.88      0.87      0.87      3000



In [19]:
# logistic regression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train_b)

lr_pred = lr.predict(X_test)

# Evaluate the classifier
print("Confusion Matrix:")
print(confusion_matrix(y_test_b, lr_pred))
print("\nClassification Report:")
print(classification_report(y_test_b, lr_pred))

NameError: name 'y_train_b' is not defined

In [ ]:
# random forest
from sklearn.ensemble import RandomForestClassifier

# Create and train the model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train_b)

# Predict and evaluate
rf_pred = rf.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test_b, rf_pred))
print("\nClassification Report:")
print(classification_report(y_test_b, rf_pred))


Confusion Matrix:
[[2717   14]
 [ 228   41]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.96      2731
           1       0.75      0.15      0.25       269

    accuracy                           0.92      3000
   macro avg       0.83      0.57      0.61      3000
weighted avg       0.91      0.92      0.89      3000



In [ ]:
# support vector machines
from sklearn.svm import SVC

# Create and train the model
svm = SVC(kernel='linear')
svm.fit(X_train, y_train_b)

# Predict and evaluate
svm_pred = svm.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test_b, svm_pred))
print("\nClassification Report:")
print(classification_report(y_test_b, svm_pred))


Confusion Matrix:
[[2731    0]
 [ 269    0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      2731
           1       0.00      0.00      0.00       269

    accuracy                           0.91      3000
   macro avg       0.46      0.50      0.48      3000
weighted avg       0.83      0.91      0.87      3000



c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\micha\anaconda3\envs\trauma\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# to add: ridge regression, PCA